Sensor Stream  →  Per-Engine Buffer  →  LSTM  →  RUL Prediction
                     (last 30 cycles)


{
  "buffer": deque(maxlen=WINDOW),
  "last_prediction": float
}


In [1]:
import torch
import pickle
from collections import deque
import numpy as np

# --------------------
# 1. Load artifacts
# --------------------
MODEL_PATH = "rul_lstm_fd001.pth"
SCALER_PATH = "scaler_fd001.pkl"
FEATURES_PATH = "features_fd001.pkl"
ARCH_PATH = "model_arch_fd001.pkl"

with open(SCALER_PATH, "rb") as f:
    scaler = pickle.load(f)

with open(FEATURES_PATH, "rb") as f:
    feature_cols = pickle.load(f)

with open(ARCH_PATH, "rb") as f:
    meta = pickle.load(f)

# --------------------
# 2. Recreate model
# --------------------
class RULLSTM(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        return self.fc(h_n[-1])

device = "cuda" if torch.cuda.is_available() else "cpu"

model = RULLSTM(input_dim=meta["input_dim"], hidden_dim=meta["hidden_dim"]).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

window = meta["window"]


c:\Users\91903\AppData\Local\Programs\Python\Python311\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
C:\Users\91903\AppData\Local\Temp\ipykernel_23904\3007994451.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the de

In [ ]:
from collections import deque
import numpy as np
import torch

class RULStreamPredictor:
    def __init__(self, model, scaler, feature_cols, window, device):
        self.model = model
        self.scaler = scaler
        self.feature_cols = feature_cols
        self.window = window
        self.device = device
        self.buffers = {}

        self.model.eval()

    def process_cycle(self, engine_id, raw_features):
        """
        raw_features: dict {feature_name: value}
        returns: RUL prediction or None
        """

        # init buffer
        if engine_id not in self.buffers:
            self.buffers[engine_id] = deque(maxlen=self.window)

        # feature vector in correct order
        x = np.array([raw_features[c] for c in self.feature_cols]).reshape(1, -1)

        # normalize using TRAIN scaler
        x = self.scaler.transform(x)[0]

        # append to buffer
        self.buffers[engine_id].append(x)

        # not enough history yet
        if len(self.buffers[engine_id]) < self.window:
            return None

        # prepare tensor
        window_data = np.array(self.buffers[engine_id]).reshape(1, self.window, -1)
        xt = torch.tensor(window_data, dtype=torch.float32).to(self.device)

        with torch.no_grad():
            rul_pred = self.model(xt).cpu().numpy().item()

        return max(rul_pred, 0.0)


In [ ]:
streamer = RULStreamPredictor(
    model=model,
    scaler=scaler,
    feature_cols=feature_cols,
    window=window,
    device=device
)

rul_predictions = {}

for engine_id, engine_df in test_df.groupby("engine_id"):
    engine_df = engine_df.sort_values("cycle")

    for _, row in engine_df.iterrows():
        raw_features = row[FEATURE_COLS].to_dict()

        pred = streamer.process_cycle(engine_id, raw_features)

        if pred is not None:
            rul_predictions.setdefault(engine_id, []).append(pred)


In [ ]:
final_preds = np.array([
    rul_predictions[eid][-1] for eid in sorted(rul_predictions.keys())
])


In [ ]:
import matplotlib.pyplot as plt

eid = list(rul_predictions.keys())[0]

plt.figure(figsize=(8,4))
plt.plot(rul_predictions[eid], label="Predicted RUL")
plt.xlabel("Cycle")
plt.ylabel("RUL")
plt.title(f"Engine {eid} – Streaming RUL Prediction")
plt.legend()
plt.show()


In [ ]:
ALERT_THRESHOLD = 20

def maintenance_alert(rul):
    if rul < ALERT_THRESHOLD:
        return "⚠️ MAINTENANCE REQUIRED"
    return None


In [ ]:
alert = maintenance_alert(pred)
if alert:
    print(f"Engine {engine_id}: {alert} (RUL={pred:.1f})")
